In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [12]:
%%sql
SELECT *
  FROM facts
LIMIT 5 ;
 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:


name - The name of the country.

area - The total land and sea area of the country.

population - The country's population.

population_growth- The country's population growth as a percentage.

birth_rate - The country's birth rate, or the number of births a year per 1,000 people.

death_rate - The country's death rate, or the number of death a year per 1,000 people.

area- The country's total area (both land and water).

area_land - The country's land area in square kilometers.

area_water - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.

In [18]:
%%sql
SELECT MIN(population) MINPOP ,MAX(population) MAXPOP, MIN(population_growth) MINPOP_GROWTH , MAX(population_growth) MAXPOP_GROWTH
FROM facts;

 * sqlite:///factbook.db
Done.


MINPOP,MAXPOP,MINPOP_GROWTH,MAXPOP_GROWTH
0,7256490011,0.0,4.02


lets use subqueries to zoom in on just these countries without using the specific values.

In [23]:
%%sql
SELECT name MINPOP_COUNTRY,(SELECT name MAXPOP_COUNTRY
                           FROM facts
                           WHERE population = (SELECT MAX(population)
                                              FROM facts))  AS MAXPOP_COUNTRY
FROM facts
WHERE population = (SELECT MIN(population)
                   FROM facts)

 * sqlite:///factbook.db
Done.


MINPOP_COUNTRY,MAXPOP_COUNTRY
Antarctica,World


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

In [28]:
%%sql
SELECT MIN(population) MINPOP ,MAX(population) MAXPOP, MIN(population_growth) MINPOP_GROWTH , MAX(population_growth) MAXPOP_GROWTH
FROM facts
WHERE name!= 'World';

 * sqlite:///factbook.db
Done.


MINPOP,MAXPOP,MINPOP_GROWTH,MAXPOP_GROWTH
0,1367485388,0.0,4.02


 summary statistics you found earlier while excluding the row for the whole world is the above

In [29]:
%%sql
SELECT AVG(population) AVGPOP , AVG(area) AVGAREA
FROM facts;

 * sqlite:///factbook.db
Done.


AVGPOP,AVGAREA
62094928.32231405,555093.546184739


now,we will write a query that finds all countries meeting both of the following conditions:

  The population is above average.
    
  The area is below average.

In [31]:
%%sql
SELECT name COUN_ABOVE_AVG
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts) AND population >(SELECT AVG(area)
                                                FROM facts)AND name!='World'

 * sqlite:///factbook.db
Done.


COUN_ABOVE_AVG
Bangladesh
Brazil
China
"Congo, Democratic Republic of the"
Egypt
Ethiopia
France
Germany
India
Indonesia


In [37]:
%%sql
SELECT *
FROM facts
LIMIT 3


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92


What country has the most people? What country has the highest growth rate?

In [38]:
%%sql
SELECT name HIGHPOPCOUNTRY
FROM facts
WHERE population = (SELECT MAX(population)
                   FROM facts
                   WHERE name!='World')


 * sqlite:///factbook.db
Done.


HIGHPOPCOUNTRY
China


Which countries have the highest ratios of water to land? Which countries have more water than land?

In [64]:
%%sql 
SELECT name COUNHIGHRATIO , ROUND(CAST(area_water AS FLOAT) / area_land ,4)RATIO
FROM facts
ORDER BY RATIO DESC
LIMIT 3;


 * sqlite:///factbook.db
Done.


COUNHIGHRATIO,RATIO
British Indian Ocean Territory,905.6667
Virgin Islands,4.5202
Puerto Rico,0.5548


In [50]:
%%sql
SELECT name COUNTRYHIGHWATER
FROM facts
WHERE area_water=(SELECT MAX(area_water)
                 FROM facts)

 * sqlite:///factbook.db
Done.


COUNTRYHIGHWATER
Canada


Which country will add the most people to their population next year?

In [52]:
%%sql 
SELECT name COUNTRYADDPOP
FROM facts
WHERE birth_rate= (SELECT MAX(birth_rate)
                 FROM facts)

 * sqlite:///factbook.db
Done.


COUNTRYADDPOP
Niger


Which countries have a higher death rate than birth rate?

In [66]:
%%sql
SELECT name COUNHIGHDEATH
FROM facts
WHERE death_rate>birth_rate;

 * sqlite:///factbook.db
Done.


COUNHIGHDEATH
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


What countries have the highest population/area ratio?

In [73]:
%%sql
SELECT name HIGHCOUN_POPAREA_RATIO
FROM facts
WHERE name!='World'
ORDER BY (population/area) DESC 
LIMIT 20;

 * sqlite:///factbook.db
Done.


HIGHCOUN_POPAREA_RATIO
Macau
Monaco
Singapore
Hong Kong
Gaza Strip
Gibraltar
Bahrain
Maldives
Malta
Bermuda
